In [66]:
# importing packages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from tqdm.notebook import tqdm
import datetime
import itertools 
import warnings
warnings.simplefilter("ignore")

import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, optimizers, applications
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import MaxPool2D, Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import VGG19, MobileNetV2
from tensorflow.keras.metrics import AUC, Precision, Recall

from sklearn import metrics
from sklearn.metrics import confusion_matrix, auc, roc_curve, precision_recall_curve

%reload_ext autoreload
%autoreload 2
from utils import *

# VGG16-based CNN Model

## Generating the Datasets

In [25]:
# create datasets for training, validation, and testing
train_fldr = './split/train'
val_fldr = './split/val'
test_fldr = './split/test'

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_fldr,
        target_size = (256, 256),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_fldr, 
        target_size = (256, 256),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_fldr, 
        target_size = (256, 256),
        batch_size = 1,
        class_mode = 'binary',
        shuffle = False,
        seed = 42)

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
print(f'STEP_SIZE_TRAIN: {STEP_SIZE_TRAIN}')
print(f'STEP_SIZE_VALID: {STEP_SIZE_VALID}')
print(f'STEP_SIZE_TEST: {STEP_SIZE_TEST}')

Found 1599 images belonging to 2 classes.
Found 401 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
STEP_SIZE_TRAIN: 99
STEP_SIZE_VALID: 25
STEP_SIZE_TEST: 600


## Constructing the Model

In [29]:
# build model similar to VGG16
def vgg16(input_shape):
    return Sequential([
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),   
        Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'),  
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),   
        Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'), 
        Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),   
        Flatten(),
        Dense(units=4096, activation='relu'),
        Dense(units=1, activation='sigmoid')
        ])

model_1 = vgg16((256, 256, 3))

In [30]:
model_1.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 256, 256, 64)      1792      
                                                                 
 conv2d_44 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 batch_normalization_15 (Bat  (None, 256, 256, 64)     256       
 chNormalization)                                                
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 128, 128, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_45 (Conv2D)          (None, 128, 128, 128)     73856     
                                                                 
 conv2d_46 (Conv2D)          (None, 128, 128, 128)    

In [31]:
# compile model
model_1.compile(optimizer = Adam(lr=0.001), 
               loss = 'binary_crossentropy',
               metrics = ['accuracy', AUC(), AUC(curve='PR'), Precision(), Recall()])

In [32]:
# Define callbacks
lr_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                 patience = 3,
                                 verbose = 1,
                                 factor = 0.5,
                                 min_lr = 0.00001)
filepath = "./logs/log_vgg16/model.{epoch:02d}-{val_loss:.2f}.hdf5"
mcp_save = ModelCheckpoint(filepath, 
                           verbose = 1, 
                           monitor = 'val_loss', 
                           mode = 'min')
csv_logger = CSVLogger('./logs/log_vgg16/log.csv')
log_dir = "./logs/log_vgg16/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir = log_dir,
                             histogram_freq = 1,
                             update_freq = 'batch')

## Training the Model

In [18]:
# train model
history_1 = model_1.fit_generator(generator = train_generator,
                               steps_per_epoch = STEP_SIZE_TRAIN,
                               validation_data = valid_generator,
                               validation_steps = STEP_SIZE_VALID,
                               epochs = 16,
                               callbacks = [lr_reduction, mcp_save, tensorboard_cb, csv_logger])

Epoch 1/15
17/99 [====>.........................] - ETA: 41:38 - loss: 38.1820 - accuracy: 0.6581 - auc_2: 0.5179 - auc_3: 0.8154 - precision_1: 0.8159 - recall_1: 0.7455

# Simplified CNN Model

## Generating the Datasets

In [33]:
# Set path for trasining testing and validation
# Data Generator for training, validation, and testing
train_fldr = './split/train'
val_fldr = './split/val'
test_fldr = './split/test'

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_fldr,
        target_size = (256, 256),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_fldr, 
        target_size = (256, 256),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_fldr, 
        target_size = (256, 256),
        batch_size = 1,
        class_mode = 'binary',
        shuffle = False,
        seed = 42)

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
print(f'STEP_SIZE_TRAIN: {STEP_SIZE_TRAIN}')
print(f'STEP_SIZE_VALID: {STEP_SIZE_VALID}')
print(f'STEP_SIZE_TEST: {STEP_SIZE_TEST}')

Found 1599 images belonging to 2 classes.
Found 401 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
STEP_SIZE_TRAIN: 99
STEP_SIZE_VALID: 25
STEP_SIZE_TEST: 600


## Constructing the Model

In [34]:
# construct model
def cnn(input_shape):
    return Sequential([
        Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
        Dropout(0.2),
        MaxPool2D(pool_size=(3, 3)),
        Conv2D(filters=32, kernel_size=(2, 2), activation='relu', padding='same'),
        Dropout(0.2),
        MaxPool2D(pool_size=(2, 2)),
        Conv2D(filters=64, kernel_size=(2, 2), activation='relu', padding='same'),
        Dropout(0.2),
        MaxPool2D(pool_size=(2, 2)),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        Dropout(0.2), 
        MaxPool2D(pool_size=(3, 3)),
        Flatten(),
        Dense(units=32, activation='relu'),
        Dropout(0.25),     
        Dense(units=1, activation='sigmoid')
        ])

model_2 = cnn((256, 256, 3))

In [35]:
model_2.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_53 (Conv2D)          (None, 256, 256, 16)      448       
                                                                 
 dropout_16 (Dropout)        (None, 256, 256, 16)      0         
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 85, 85, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 85, 85, 32)        2080      
                                                                 
 dropout_17 (Dropout)        (None, 85, 85, 32)        0         
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 42, 42, 32)       0         
 g2D)                                                 

In [36]:
# compile model
model_2.compile(optimizer = Adam(lr=0.001), 
                loss = 'binary_crossentropy',
                metrics = ['accuracy', AUC(), AUC(curve='PR'), Precision(), Recall()])

In [37]:
# define callbacks
lr_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                 patience = 3,
                                 verbose = 1,
                                 factor = 0.5,
                                 min_lr = 0.00001)
filepath = "./logs/log_cnn/model.{epoch:02d}-{val_loss:.2f}.hdf5"
mcp_save = ModelCheckpoint(filepath, 
                           verbose = 1, 
                           monitor = 'val_loss', 
                           mode = 'min')
csv_logger = CSVLogger('./logs/log_cnn/log.csv')
log_dir = "./logs/log_cnn/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir = log_dir,
                             histogram_freq = 1,
                             update_freq = 'batch')

## Training the Model

In [22]:
# train model
history_2 = model_2.fit_generator(generator = train_generator,
                                  steps_per_epoch = STEP_SIZE_TRAIN,
                                  validation_data = valid_generator,
                                  validation_steps = STEP_SIZE_VALID,
                                  epochs = 9,
                                  callbacks = [lr_reduction, mcp_save, tensorboard_cb, csv_logger])

# ResNet50 Architecture

## Generating the Datasets

In [38]:
# Set path for training testing and validation
# Data Generator for training, validation, and testing
train_fldr = './split/train'
val_fldr = './split/val'
test_fldr = './split/test'

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_fldr,
        target_size = (256, 256),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_fldr, 
        target_size = (256, 256),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_fldr, 
        target_size = (256, 256),
        batch_size = 1,
        class_mode = 'binary',
        shuffle = False,
        seed = 42)

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
print(f'STEP_SIZE_TRAIN: {STEP_SIZE_TRAIN}')
print(f'STEP_SIZE_VALID: {STEP_SIZE_VALID}')
print(f'STEP_SIZE_TEST: {STEP_SIZE_TEST}')

Found 1599 images belonging to 2 classes.
Found 401 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
STEP_SIZE_TRAIN: 99
STEP_SIZE_VALID: 25
STEP_SIZE_TEST: 600


## Constructing the Model

In [48]:
# model_3 creation
cnn_base = ResNet50(include_top = False,
                    weights = "imagenet",
                    input_shape = (256, 256, 3))
model_3 = models.Sequential()
model_3.add(cnn_base)
model_3.add(Flatten())
model_3.add(Dense(units = 64, activation = 'relu'))
model_3.add(Dropout(0.3))
model_3.add(Dense(units = 128, activation = 'relu'))
model_3.add(Dropout(0.3))
model_3.add(layers.Dense(units = 256, activation = 'relu'))
model_3.add(Dropout(0.3))
model_3.add(layers.Dense(units = 128, activation = 'relu'))
model_3.add(Dropout(0.3))
model_3.add(layers.Dense(units = 1, activation = 'sigmoid'))

cnn_base.trainable = False

In [51]:
model_3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten_9 (Flatten)         (None, 131072)            0         
                                                                 
 dense_21 (Dense)            (None, 64)                8388672   
                                                                 
 dropout_25 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 128)               8320      
                                                                 
 dropout_26 (Dropout)        (None, 128)               0         
                                                                 
 dense_23 (Dense)            (None, 256)              

In [49]:
# compile model
model_3.compile(loss = 'binary_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy', tf.metrics.Precision(), tf.metrics.Recall(), tf.metrics.AUC(), tf.metrics.AUC(curve='PR')])

In [79]:
# define callbacks
earlyStopping = EarlyStopping(monitor = 'val_loss', 
                              verbose = 1, 
                              mode = 'min', 
                              patience = 4)
lr_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                 patience = 3,
                                 verbose = 1,
                                 factor = 0.5,
                                 min_lr = 0.00001)
filepath = "./logs/log_resnet/model.{epoch:02d}-{val_loss:.2f}.hdf5"
mcp_save = ModelCheckpoint(filepath, 
                           verbose = 1, 
                           monitor = 'val_loss', 
                           mode = 'min')
log_dir = "./logs/log_resnet/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir=log_dir,
                             histogram_freq = 1,
                             update_freq = 'batch')
csv_logger = CSVLogger('./logs/log_resnet/log.csv')

## Training the Model

In [ ]:
# train model
history_3 = model_3.fit_generator(generator = train_generator,
                             steps_per_epoch = STEP_SIZE_TRAIN,
                             validation_data = valid_generator,
                             validation_steps = STEP_SIZE_VALID,
                             epochs = 15,
                             callbacks = [lr_reduction, earlyStopping, mcp_save, tensorboard_cb, csv_logger])

# AlexNet Architecture

## Generating the Datasets

In [47]:
# Set path for trasining testing and validation
# Data Generator for training, validation, and testing
train_fldr = './split/train'
val_fldr = './split/val'
test_fldr = './split/test'

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_fldr,
        target_size = (227, 227),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_fldr, 
        target_size = (227, 227),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_fldr, 
        target_size = (227, 227),
        batch_size = 1,
        class_mode = 'binary',
        shuffle = False,
        seed = 42)

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
print(f'STEP_SIZE_TRAIN: {STEP_SIZE_TRAIN}')
print(f'STEP_SIZE_VALID: {STEP_SIZE_VALID}')
print(f'STEP_SIZE_TEST: {STEP_SIZE_TEST}')

Found 1599 images belonging to 2 classes.
Found 401 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
STEP_SIZE_TRAIN: 99
STEP_SIZE_VALID: 25
STEP_SIZE_TEST: 600


## Constructing the Model

In [52]:
# construct model
model_4 = Sequential([
    Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [53]:
model_4.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_57 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization_20 (Bat  (None, 55, 55, 96)       384       
 chNormalization)                                                
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_58 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_21 (Bat  (None, 27, 27, 256)      1024      
 chNormalization)                                                
                                                     

In [54]:

# compile model
model_4.compile(loss = 'binary_crossentropy',
               optimizer = Adam(lr = 0.001),
               metrics = ['accuracy', Precision(), Recall(), AUC(), AUC(curve='PR')])

In [56]:
# define callbacks
lr_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                 patience = 3,
                                 verbose = 1,
                                 factor = 0.5,
                                 min_lr = 0.00001)
filepath = "./logs/log_alexnet_2/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
mcp_save = ModelCheckpoint(filepath, 
                           verbose = 1, 
                           monitor = 'val_loss', 
                           mode = 'min')
log_dir = "./logs/log_alexnet_2/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir=log_dir,
                             histogram_freq = 1,
                             update_freq = 'batch')
csv_logger = CSVLogger('./logs/log_alexnet_2/log.csv')

## Training the Model

In [ ]:
# train model
history_4 = model_4.fit_generator(generator = train_generator,
                               steps_per_epoch = STEP_SIZE_TRAIN,
                               validation_data = valid_generator,
                               validation_steps = STEP_SIZE_VALID,
                               epochs = 12,
                               callbacks = [lr_reduction, mcp_save, tensorboard_cb, csv_logger])

# VGG19 Architecture

## Generating the Datasets

In [58]:
# generate datasets
train_fldr = './split/train'
val_fldr = './split/val'
test_fldr = './split/test'

train_generator = ImageDataGenerator(rescale = 1./255).flow_from_directory(
        train_fldr,
        target_size = (224, 224),
        batch_size = 10,
        class_mode = 'binary',
        seed = 42)
valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_fldr, 
        target_size = (224, 224),
        batch_size = 10,
        class_mode = 'binary',
        seed = 42)

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_fldr, 
        target_size = (224, 224),
        class_mode = 'binary',
        batch_size = 1,
        seed = 42,
        shuffle = False)

STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
print(f'STEP_SIZE_TRAIN: {STEP_SIZE_TRAIN}')
print(f'STEP_SIZE_VALID: {STEP_SIZE_VALID}')
print(f'STEP_SIZE_TEST: {STEP_SIZE_TEST}')

Found 1599 images belonging to 2 classes.
Found 401 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
STEP_SIZE_TRAIN: 159
STEP_SIZE_VALID: 40
STEP_SIZE_TEST: 600


## Constructing the Model

In [59]:
# construct model
vgg_19 = VGG19(include_top = False, 
               weights = 'imagenet', 
               input_shape = (224, 224, 3))

vgg_19.trainable = False

model_5 = Sequential([
    vgg_19,
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(1, activation='sigmoid')
])

80150528/80134624 [==============================] - 6s 0us/step


In [60]:
model_5.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_11 (Flatten)        (None, 25088)             0         
                                                                 
 dense_29 (Dense)            (None, 4096)              102764544 
                                                                 
 dense_30 (Dense)            (None, 4096)              16781312  
                                                                 
 dense_31 (Dense)            (None, 1)                 4097      
                                                                 
Total params: 139,574,337
Trainable params: 119,549,953
Non-trainable params: 20,024,384
_________________________________________________________________


In [63]:
# compile model
model_5.compile(optimizer = Adam(lr = 0.001),
                loss = 'binary_crossentropy',
                metrics = ['accuracy', AUC(), AUC(curve='PR'), Precision(), Recall()])

In [62]:
lr_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                 patience = 3,
                                 verbose = 1,
                                 factor = 0.5,
                                 min_lr = 0.00001)
filepath = "./logs/log_vgg19/model.{epoch:02d}-{val_loss:.2f}.hdf5"
mcp_save = ModelCheckpoint(filepath, 
                           verbose = 1, 
                           monitor = 'val_loss', 
                           mode = 'min')
csv_logger = CSVLogger('./logs/log_vgg19/log.csv')
log_dir = "./logs/log_vgg19/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir = log_dir,
                             histogram_freq = 1,
                             update_freq = 'batch')

## Training the Model

In [ ]:
# train model
history_5 = model_5.fit_generator(generator = train_generator,
                                  steps_per_epoch = STEP_SIZE_TRAIN,
                                  validation_data = valid_generator,
                                  validation_steps = STEP_SIZE_VALID,
                                  epochs = 10,
                                  callbacks = [lr_reduction, mcp_save, tensorboard_cb, csv_logger])

# MobileNet Architecture

## Generating the Datasets

In [72]:
# create datasets for training, validation, and testing
train_fldr = './split/train'
val_fldr = './split/val'
test_fldr = './split/test'

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_fldr,
        target_size = (224, 224),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_fldr, 
        target_size = (224, 224),
        batch_size = 16,
        class_mode = 'binary',
        seed = 42)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_fldr, 
        target_size = (224, 224),
        batch_size = 1,
        class_mode = 'binary',
        shuffle = False,
        seed = 42)

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
print(f'STEP_SIZE_TRAIN: {STEP_SIZE_TRAIN}')
print(f'STEP_SIZE_VALID: {STEP_SIZE_VALID}')
print(f'STEP_SIZE_TEST: {STEP_SIZE_TEST}')

Found 1599 images belonging to 2 classes.
Found 401 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
STEP_SIZE_TRAIN: 99
STEP_SIZE_VALID: 25
STEP_SIZE_TEST: 600


## Constructing the Model

In [73]:
# create base model for transfer learning
base_model = MobileNetV2(
    input_shape = (224, 224, 3),
    include_top = False, 
    weights = 'imagenet'
)

In [74]:
# freezes all neurons for base model
base_model.trainable = False 
# construct model
model_6 = Sequential([ 
    base_model,
    Conv2D(32, 3, activation = 'relu'), 
    Dropout(0.2), 
    GlobalAveragePooling2D(), 
    Dense(1, activation = 'sigmoid')
])

In [75]:
model_6.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d_63 (Conv2D)          (None, 5, 5, 32)          368672    
                                                                 
 dropout_32 (Dropout)        (None, 5, 5, 32)          0         
                                                                 
 global_average_pooling2d_1   (None, 32)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_33 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,626,689
Trainable params: 368,705
Non

In [76]:
# compile model
model_6.compile(
    optimizer = Adam(lr = 0.001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy', Precision(), Recall(), AUC(), AUC(curve='PR')])

In [77]:
# Define callbacks
lr_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                 patience = 3,
                                 verbose = 1,
                                 factor = 0.5,
                                 min_lr = 0.00001)
filepath = "./logs/log_mobilenet/model.{epoch:02d}-{val_loss:.2f}.hdf5"
mcp_save = ModelCheckpoint(filepath, 
                           verbose = 1, 
                           monitor = 'val_loss', 
                           mode = 'min')
csv_logger = CSVLogger('./logs/log_mobilenet/log.csv')
log_dir = "./logs/log_mobilenet/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir = log_dir,
                             histogram_freq = 1,
                             update_freq = 'batch')

## Training the Model

In [78]:
# train model
history = model_6.fit_generator(generator = train_generator,
                             steps_per_epoch = STEP_SIZE_TRAIN,
                             validation_data = valid_generator,
                             validation_steps = STEP_SIZE_VALID,
                             epochs = 3,
                             callbacks = [lr_reduction, mcp_save, tensorboard_cb, csv_logger])

Epoch 1/3
99/99 [==============================] - ETA: 0s - loss: 0.5042 - accuracy: 0.8086 - precision_12: 0.8118 - recall_12: 0.9945 - auc_24: 0.6321 - auc_25: 0.8686
Epoch 00001: saving model to ../logs/log_mobilenet/model.01-0.41.hdf5
99/99 [==============================] - 358s 4s/step - loss: 0.5042 - accuracy: 0.8086 - precision_12: 0.8118 - recall_12: 0.9945 - auc_24: 0.6321 - auc_25: 0.8686 - val_loss: 0.4073 - val_accuracy: 0.8125 - val_precision_12: 0.8125 - val_recall_12: 1.0000 - val_auc_24: 0.7964 - val_auc_25: 0.9377 - lr: 0.0010
Epoch 2/3
99/99 [==============================] - ETA: 0s - loss: 0.4051 - accuracy: 0.8143 - precision_12: 0.8143 - recall_12: 1.0000 - auc_24: 0.7822 - auc_25: 0.9331
Epoch 00002: saving model to ../logs/log_mobilenet/model.02-0.39.hdf5
99/99 [==============================] - 335s 3s/step - loss: 0.4051 - accuracy: 0.8143 - precision_12: 0.8143 - recall_12: 1.0000 - auc_24: 0.7822 - auc_25: 0.9331 - val_loss: 0.3946 - val_accuracy: 0.8125 